In [1]:
# AUSTESTEN DER BERECHNUNG DER PARAMETER DES OBJEKTES AUS EINEM BILD #
# UND #
# BERECHNUNG VON FIGUREN DIE SPEZIFISCH FÜR CAGING UND PUSHING GEHEN ODER NUR FÜR EINES DER BEIDEN #

In [2]:
import cv2 as cv
import numpy as np

def shape_to_image(shape):
    x = shape[0]
    y = shape[1]
    # shape to opencv image
    scale = 100
    frame = 10/scale

    # Alle Punkte in den positiven bereich verschieben
    x = np.add(x, np.min(x) * -1 + frame) * scale
    y = np.add(y, np.min(y) * -1 + frame) * scale

    # leeres Bild erstellen
    img = np.zeros((int(np.max(y) + frame * scale), int(np.max(x) + frame * scale)), dtype=np.uint8)

    # Zeichne alle Punkte in das Bild
    for point in zip(*[x, y]):
        cv.circle(img, (int(point[0]), int(point[1])), 1, 255, -1)

    return img

shape = [[-2.0,-1.0,1,2,3,4,5],[-2,-1,1,1,1,1,1]]
image = shape_to_image(shape)
cv.imwrite("TestBild.jpeg", image)

True

In [3]:
import cv2
import numpy as np
import imutils

def rectify_img(img):
    # Definiere den Rotationswinkel im Uhrzeigersinn
    angle = 14.8 # 14.8 # 199
    # Definiere die Streckung
    strech = 0.526 # 0.526

    # Bestimme die Abmessungen des Bildes
    height, width = img.shape[:2]

    # Berechne die Rotationsmatrix
    M = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)

    # Wende die affine Transformation auf das Bild an
    img = cv2.warpAffine(img, M, (width, height))

    # Bestimme die neue Höhe des Bildes nach der Streckung in der Y-Richtung
    new_height = int(height * strech)

    # Skaliere das Bild auf die neue Höhe
    img = cv2.resize(img, (width, new_height))

    # Bild wieder zurück drehen
    M = cv2.getRotationMatrix2D((width/2, new_height/2), 360 - angle, 1)
    img = cv2.warpAffine(img, M, (width, new_height))

    cv2.imwrite("Entzert.jpeg", img)
    return img

In [4]:

def get_contours(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    for i in range(4):
        # Bild glätten
        median = cv2.medianBlur(gray, 5)
        blurred = cv2.GaussianBlur(median, (5, 5), 0)
        
        # Lücken schließen
        kernel = np.ones((5,5),np.uint8)
        closing = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        
        # Schwellenwert anwenden
        thresh = cv2.threshold(opening, 1, 255, cv2.THRESH_BINARY)[1]
        gray = thresh
        
    edges = cv2.Canny(thresh,100,200)
    cv2.imwrite("edges.jpeg", edges)
    
    # Konturen extrahieren
    contours = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    return contours

def save_rect_approx_img(rect,approx,img):
    box = np.intp(cv2.boxPoints(rect))
    cv2.drawContours(img,[box],0,(0, 255, 0),2)
    cv2.drawContours(img, [approx], -1, (0, 0, 255), 2)
    cv2.imwrite("Ausgewertet.jpeg", img)

def get_approx_dmin(img):
    contours = get_contours(img)
    w = 0.0
    h = 0.0
    rect = None
    approx = None
    print("len(contours) =", len(contours))
    
    for contour in contours:
        # Umgebendes Rechteck berechnen
        rect_new = cv2.minAreaRect(contour)
        w_new = rect_new[1][0]
        h_new = rect_new[1][1]

    if(w_new * h_new > w * h):
        w = w_new
        h = h_new
        rect = rect_new
        approx = cv2.approxPolyDP(contour, 15, True)
        
    dmin = min(w, h)
        
    print("w =",w,"cm  h =", h, "cm")
    save_rect_approx_img(rect,approx,img)

    return approx,dmin




In [5]:

def get_linepoints(p1,p2):
    # Bestimmung der Steigung
    if p2[0] - p1[0] != 0:
        m = (p2[1] - p1[1]) / (p2[0] - p1[0])
    else:
        m = np.sign(p2[1] - p1[1]) * 9999999999999999

    # Bestimmung des Y-Achsenabschnitts
    b = p1[1] - m * p1[0]

    # Bestimmung des größeren und kleineren Punkts
    if p1[0] < p2[0]:
        x1 = p1[0]
        x2 = p2[0]
    else:
        x1 = p2[0]
        x2 = p1[0]

    line_points = []
    for x in range(int(x1), int(x2) + 1):
        y = m * x + b
        line_points.append([x, y])
    return line_points

def approx2contour(approx):
    contour = []
    for i in range(len(approx)-1):
        line_points = get_linepoints(approx[i][0],approx[i+1][0])
        contour.extend(line_points)
    contour = np.array(contour).reshape((-1, 1, 2)).astype(np.float32)
    return contour

def pt2cvpt(point):
    return(int(point[0]),int(point[1]))

def get_dmax(contour):
    d_max = 0.0
    max_line = []

    for i in range(len(contour)):    
        for j in range(i,len(contour)):
            p1 = contour[i][0]
            p2 = contour[j][0]
            dist = np.linalg.norm(p1 - p2)
            if dist > d_max:
                d_max = dist
                max_line = [pt2cvpt(p1),pt2cvpt(p2)]
                    
    # draw d_max in img
    img = cv2.imread("Ausgewertet.jpeg")
    cv2.line(img, max_line[0], max_line[1], (255, 255, 0), 2)
    cv2.imwrite("Ausgewertet.jpeg", img)
        
    return d_max



In [6]:
image_path = 'Shape.jpeg'
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img = rectify_img(img)
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
approx, d_min = get_approx_dmin(img)
convex = cv2.isContourConvex(approx)
area = cv2.contourArea(approx)
contour = approx2contour(approx)
d_max = get_dmax(contour)


# img = cv2.imread(image_path)
# contour = np.squeeze(contour)
# print(contour.shape)
# print(contour.dtype)
# cv2.drawContours(img, [contour], -1, (0, 0, 255), 2)
# cv2.imwrite("Contour.jpeg", img)

print("d_min =", d_min / 100, "m")
print("d_max =", d_max / 100, "m") 
print("area =", area / 10000, "m²") 
print("convex =", convex)

len(contours) = 1
w = 421.7337646484375 cm  h = 305.0216979980469 cm
d_min = 3.0502169799804686 m
d_max = 4.467527160644531 m
area = 8.96575 m²
convex = False


In [7]:
import numpy as np

# Berechnung des Gewichts
density = 0.8
r = 1.25
h = 0.2
a = 2
b = 2
area = np.pi * r**2
area = a * b
#area = (a**2 * np.sqrt(3)) / 4
weight = area * h * density
print("weight =",weight, "kg")


d_min = 3.66
d_max = 4.0
e = d_max * 0.076
r_cage = 0.5 * d_max + 0.220 + 0.20925 + e
min_robots = (2 * np.pi * r_cage) / (2 * 0.220 + d_min)
print("min_robots =", min_robots)

weight = 0.6400000000000001 kg
min_robots = 4.188662497767951


In [8]:
import numpy as np

def calculate_min_robots(d_min, d_max):
    e = d_max * 0.076
    r_cage = 0.5 * d_max + 0.220 + 0.20925 + e
    min_robots = (2 * np.pi * r_cage) / (2 * 0.220 + d_min)
    return min_robots


best_d_min = 0.0
best_d_max = 0.0
best_min_robots = 0.0

for a in np.arange(0.1, 10.0, 0.1):
    for b in np.arange(a, 10.0, 0.1):
        for c in np.arange(b, 10.0, 0.1):
            d_min = min(a, b, c)
            d_max = max(a, b, c)
            robots = calculate_min_robots(d_min, d_max)
            if d_max > best_d_max and robots < 4.5:
                best_a = a
                best_b = b
                best_c = c
                best_min_robots = robots
            

print("Größtmögliches Dreieck:")
print("min_robots =", best_min_robots)
print("a =", best_a)
print("b =", best_b)
print("c =", best_c)

Größtmögliches Dreieck:
min_robots = 3.7259471169020997
a = 9.9
b = 9.9
c = 9.9


In [9]:
import numpy as np

def calculate_min_robots(d_min, d_max):
    e = d_max * 0.076
    r_cage = 0.5 * d_max + 0.220 + 0.20925 + e
    min_robots = (2 * np.pi * r_cage) / (2 * 0.220 + d_min)
    return min_robots


best_d_max = 0.0
best_a = 0.0
best_b = 0.0
best_min_robots = 0.0

# Iteration über verschiedene Werte von d_min und d_max
for a in np.arange(0.0, 10.0, 0.1):
    for b in np.arange(a, 10.0, 0.1):
        d_min = min(a, b)
        d_max = np.sqrt(a*a+b*b)
        robots = calculate_min_robots(d_min, d_max)
        if d_max > best_d_max and robots < 5.2:
            best_a = a
            best_b = b
            best_d_max = d_max
            best_min_robots = robots
            

print("Größtmögliches Quadrat:")
print("min_robots =", best_min_robots)
print("a =", best_a)
print("b =", best_b)

Größtmögliches Quadrat:
min_robots = 5.161242613753904
a = 9.9
b = 9.9


In [10]:
import numpy as np

def calculate_min_robots(d_min, d_max):
    e = d_max * 0.076
    r_cage = 0.5 * d_max + 0.220 + 0.20925 + e
    min_robots = (2 * np.pi * r_cage) / (2 * 0.220 + d_min)
    return min_robots


best_d_max = 0.0
best_r = 0.0
best_min_robots = 0.0

for r in np.arange(0.0, 10.0, 0.1):
    d_min = 2*r
    d_max = 2*r
    robots = calculate_min_robots(d_min, d_max)
    if d_max > best_d_max and robots < 4.5:
        best_r = r
        best_d_max = d_max
        best_min_robots = robots
            

print("Größtmöglicher Kreis:")
print("min_robots =", best_min_robots)
print("r =", best_r)

Größtmöglicher Kreis:
min_robots = 3.6736921484401477
r = 9.9
